# Class 3: Model Deployment and Ethical Considerations

## Welcome to Week 10, Class 3!
In this notebook, we’ll learn how to **deploy** an AI model so it can be used in real-world applications, and we’ll discuss **ethical considerations** in AI, such as bias and fairness. You’ll save and load a model, create a simple Flask API to serve predictions, and reflect on ethical challenges.

**Objectives**:
- Save and load trained models using pickle.
- Build a Flask API to serve model predictions.
- Understand ethical issues in AI, including bias, fairness, and transparency.
- Discuss ethics in the context of your capstone project.

**Let’s get started!**

## 1. Why Deploy Models?
Training a model is only half the battle—deploying it makes it useful. Deployment means making a model accessible, e.g., via:
- A web app (like a sentiment analyzer).
- A mobile app.
- An internal tool for a company.

We’ll use **Flask**, a lightweight Python framework, to create a simple API that takes text input and returns predictions.

**Discussion Question**: What are some real-world apps where deployed AI models are used?

## 2. Setup
Let’s install and import the required libraries. Run the cell below to set up the environment.

**Note**: Flask apps are typically run locally, not in Jupyter. We’ll write the Flask code here but test it in a separate `.py` file later.

In [ ]:
# Install libraries (uncomment if needed)
# !pip install numpy pandas scikit-learn flask joblib nltk

# Import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import warnings
warnings.filter_warnings('ignore')

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

print("Setup complete!")

## 3. Saving and Loading Models
Once a model is trained, we save it to avoid retraining. We’ll use **joblib** (similar to pickle) to save a sentiment analysis model.

Let’s train a simple model (like Class 1’s) and save it.

### 3.1 Train and Save a Model
We’ll use a toy dataset for sentiment analysis and save the trained model and vectorizer.

In [ ]:
# Toy dataset
data = {
    "review": [
        "I loved the movie it was great",
        "Terrible film so boring",
        "Amazing story and acting",
        "I hated this movie awful",
        "Really enjoyed the plot",
        "Worst movie ever"
    ],
    "sentiment": ["positive", "negative", "positive", "negative", "positive", "negative"]
}
df = pd.DataFrame(data)

# Preprocessing function
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Apply preprocessing
df["clean_review"] = df["review"].apply(preprocess_text)

# Create BoW vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["clean_review"])
y = df["sentiment"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = MultinomialNB()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save model and vectorizer
joblib.dump(model, "sentiment_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

print("Model and vectorizer saved!")

### 3.2 Load and Test the Model
Let’s load the saved model and make a prediction.

In [ ]:
# Load model and vectorizer
loaded_model = joblib.load("sentiment_model.pkl")
loaded_vectorizer = joblib.load("vectorizer.pkl")

# Test a new review
new_review = "This movie was fantastic"
clean_review = preprocess_text(new_review)
new_vector = loaded_vectorizer.transform([clean_review])
prediction = loaded_model.predict(new_vector)

print(f"Review: {new_review}\nPredicted sentiment: {prediction[0]}")

**Your Turn**: Test the loaded model on a review of your own.

In [ ]:
# Your code here
your_review = "Your review here"  # Write your own review
clean_your_review = preprocess_text(your_review)
your_vector = loaded_vectorizer.transform([clean_your_review])
your_prediction = loaded_model.predict(your_vector)

print(f"Your review: {your_review}\nPredicted sentiment: {your_prediction[0]}")

## 4. Creating a Flask API
An **API** lets users interact with your model over the web. We’ll create a Flask app that:
- Accepts a text review (via a POST request).
- Returns the predicted sentiment.

**Note**: We’ll write the Flask code here, but you’ll need to run it in a separate `.py` file (e.g., `app.py`) outside Jupyter.

Below is the Flask app code. Copy it into a file named `app.py` in the same folder as `sentiment_model.pkl` and `vectorizer.pkl`.

In [ ]:
# This is a template for app.py (don’t run in Jupyter)
print("""
from flask import Flask, request, jsonify
import joblib
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

app = Flask(__name__)

# Load model and vectorizer
model = joblib.load("sentiment_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

# NLTK setup
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    if 'review' not in data:
        return jsonify({'error': 'No review provided'}), 400
    review = data['review']
    clean_review = preprocess_text(review)
    vector = vectorizer.transform([clean_review])
    prediction = model.predict(vector)[0]
    return jsonify({'review': review, 'sentiment': prediction})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
""")

print("Copy the above code into 'app.py'.")

### 4.1 How to Run and Test the API
1. Save the code above as `app.py`.
2. Ensure `sentiment_model.pkl` and `vectorizer.pkl` are in the same folder.
3. Open a terminal, navigate to the folder, and run:
   ```bash
   python app.py
   ```
4. The API will run at `http://localhost:5000`.
5. Test it using a tool like `curl` or Python’s `requests` library (example below).

**Test the API** with this code (run in a separate notebook or script):

In [ ]:
# Test API (run this in a separate notebook/script after starting app.py)
import requests

url = "http://localhost:5000/predict"
data = {"review": "This movie was amazing"}

try:
    response = requests.post(url, json=data)
    print(response.json())
except:
    print("Make sure app.py is running!")

# Note: This won’t work in Jupyter unless app.py is running elsewhere.

**Your Turn**: After setting up `app.py`, test the API with a review of your own. Share the response.

## 5. Ethical Considerations in AI
AI models can have unintended consequences, like **bias**, **unfairness**, or **lack of transparency**. Let’s explore:

- **Bias**: Models reflect biases in data. E.g., if reviews are mostly from one group, the model may misjudge others’ sentiments.
- **Fairness**: Ensure models don’t favor certain groups (e.g., biased hiring algorithms).
- **Transparency**: Users should know how predictions are made.

**Example**: A sentiment model trained on formal reviews might misclassify slang-heavy text (e.g., "This movie slaps!" as negative).

**Case Study**: The COMPAS algorithm predicted higher recidivism risk for certain groups, raising fairness concerns.

### 5.1 Hands-On: Exploring Bias
Let’s test our model for potential bias by trying reviews with slang or unusual phrasing.

In [ ]:
# Test reviews with slang or edge cases
test_reviews = [
    "This movie slaps, so dope!",  # Positive slang
    "Film was mid, nothing special",  # Neutral slang
    "Not my cup of tea but okay"   # Ambiguous
]

for review in test_reviews:
    clean_review = preprocess_text(review)
    vector = loaded_vectorizer.transform([clean_review])
    prediction = loaded_model.predict(vector)[0]
    print(f"Review: {review}\nPredicted sentiment: {prediction}\n")

**Your Turn**: Write two reviews (e.g., one with slang, one ambiguous) and test them. Do the predictions seem fair?

In [ ]:
# Your code here
your_test_reviews = [
    "Your first review here",  # Try slang or unusual phrasing
    "Your second review here"  # Try ambiguous or tricky text
]

for review in your_test_reviews:
    clean_review = preprocess_text(review)
    vector = loaded_vectorizer.transform([clean_review])
    prediction = loaded_model.predict(vector)[0]
    print(f"Your review: {review}\nPredicted sentiment: {prediction}\n")

**Discussion**:
- Did the model misclassify any reviews? Why might that happen?
- How could bias in our toy dataset (e.g., formal language) affect predictions?
- What ethical concerns might arise in your capstone project?

## 6. Wrap-Up
You’ve learned:
- How to save and load models with joblib.
- How to create a Flask API to serve predictions.
- Key ethical issues in AI and how to spot bias.

**Homework**:
- Read about AI ethics: [AI Bias Examples](https://www.wired.com/story/how-algorithmic-bias-can-impact-your-life/).
- Brainstorm ethical considerations for your capstone project (e.g., biased data, fairness).

**Deliverables**:
- Submit this notebook with completed "Your Turn" sections.
- Include a screenshot or output of your Flask API test.
- Write a short paragraph on one ethical challenge you observed.

**Questions?** Reach out to the instructor or discuss with peers.

Great job, and see you in Class 4 for capstone project work! 🚀